In [1]:
import pandas as pd
import numpy as np

In [2]:
def extractItemCount(df, df_qTime, df_click, intervals, col_name):
    
    
    
    df_click = getTimeInterval(df_click,intervals)
    
    if 'time_interval' not in df.columns:
        df_qTime = getTimeInterval(df_qTime,intervals)
        df = df.merge(df_qTime[['user_id','time_interval']])
    
    df_click_sta = df_click[['user_id','item_id','time_interval']].groupby(by=['item_id','time_interval'],as_index=False).count()
    df_click_sta.columns = ['item_id','time_interval',col_name]
    
    df = df.merge(df_click_sta,on=['item_id','time_interval'],how='left')
    
    return df
    

In [3]:
def getTimeInterval(df,intervals):
    df['hour_minute'] = (df['datetime'].dt.hour + df['datetime'].dt.minute/60)/24

    time_interval_list = np.linspace(0,1,intervals)

    df['time_interval'] = df['hour_minute'].apply(lambda x: np.where(x<time_interval_list)[0][0]-1 )
    df['time_interval'] = (df['datetime'].dt.day - min(df['datetime'].dt.day))*intervals + df['time_interval']
    return df

In [4]:
train_path = '../../user_data/dataset/'
test_path = '../../user_data/dataset/'
header = 'underexpose'

now_phase = 9
file_name = 'recall_0531_addsim_addAA_RA'

df = pd.read_csv('../../user_data/dataset/new_recall/' + file_name + '.csv')

In [5]:
df.shape

(17383059, 20)

In [6]:
df.head()

user_id  item_id     sim  feature_0     feature_1  feature_2  feature_3  \
0       11    26711  0.0007       50.0  5.338502e+13     0.0012        1.0   
1       11    10528  0.0007       68.0  1.723291e+10     0.0036        1.0   
2       11    22042  0.0007       40.0  4.188502e+14     0.0332        1.0   
3       11    77847  0.0005       30.0  1.547299e+05     0.0168        1.0   
4       11    85475  0.0005       34.0  8.342025e+13     0.0016        1.0   

   feature_4  feature_5  feature_6  feature_7  feature_8  feature_9  \
0     0.0000   490.3025          0     0.7968   1.593768     0.8172   
1     0.0000   472.3559          0     0.8566   4.666060     0.7858   
2     0.2152   487.1326          4     0.7798   2.138867     0.8115   
3     0.0000   472.6520          0     0.8478   1.695653     0.8479   
4     0.0000   485.4152          0     0.8179   1.635859     0.8016   

   feature_10    cn_sim   hpi_sim   hdi_sim  lhn1_sim    ra_sim    aa_sim  
0    1.634576  0.004169  0.011357  0.010454  0.028477  0.008373  0.010541  
1    4.357030  0.003971  0.008911  0.004760  0.010709  0.009812  0.011679  
2    2.254222  0.001825  0.003848  0.002922  0.006242  0.005647  0.006637  
3    1.695840  0.004337  0.010767  0.005755  0.014288  0.008373  0.009541  
4    1.603376  0.004169  0.010454  0.006619  0.016596  0.009507  0.012374

In [7]:
whole_qTime = pd.DataFrame() 

for c in range(now_phase + 1):  
    #print('phase:', c)  
    click_query = pd.read_csv(test_path + header + '_test_qtime_{}_time.csv'.format(c))  
    whole_qTime = whole_qTime.append(click_query)  
    
whole_qTime = whole_qTime.reset_index(drop=True)
whole_qTime['datetime'] = pd.to_datetime(whole_qTime['datetime'])

In [8]:
whole_qTime.shape

(17448, 4)

In [9]:
whole_click = pd.DataFrame() 
for c in range(now_phase + 1):  
    #print('phase:', c)  
    click_train = pd.read_csv(train_path + header + '_train_click_{}_time.csv'.format(c))  
    click_test = pd.read_csv(test_path + header + '_test_click_{}_time.csv'.format(c))  

    all_click = click_train.append(click_test)  
    whole_click = whole_click.append(all_click)  
    
    
whole_click =  whole_click.drop_duplicates(subset=['user_id','item_id','time'],keep='last')
whole_click = whole_click.sort_values('time')
whole_click = whole_click.reset_index(drop=True)
whole_click['datetime'] = pd.to_datetime(whole_click['datetime'])


In [10]:
whole_qTime.shape

(17448, 4)

In [11]:
df = extractItemCount(df,whole_qTime,whole_click,2,'item_count_12h')

In [12]:
df.shape

(17383059, 22)

In [13]:
df.head()

user_id  item_id     sim  feature_0     feature_1  feature_2  feature_3  \
0       11    26711  0.0007       50.0  5.338502e+13     0.0012        1.0   
1       11    10528  0.0007       68.0  1.723291e+10     0.0036        1.0   
2       11    22042  0.0007       40.0  4.188502e+14     0.0332        1.0   
3       11    77847  0.0005       30.0  1.547299e+05     0.0168        1.0   
4       11    85475  0.0005       34.0  8.342025e+13     0.0016        1.0   

   feature_4  feature_5  feature_6  ...  feature_9  feature_10    cn_sim  \
0     0.0000   490.3025          0  ...     0.8172    1.634576  0.004169   
1     0.0000   472.3559          0  ...     0.7858    4.357030  0.003971   
2     0.2152   487.1326          4  ...     0.8115    2.254222  0.001825   
3     0.0000   472.6520          0  ...     0.8479    1.695840  0.004337   
4     0.0000   485.4152          0  ...     0.8016    1.603376  0.004169   

    hpi_sim   hdi_sim  lhn1_sim    ra_sim    aa_sim  time_interval  \
0  0.011357  0.010454  0.028477  0.008373  0.010541              6   
1  0.008911  0.004760  0.010709  0.009812  0.011679              6   
2  0.003848  0.002922  0.006242  0.005647  0.006637              6   
3  0.010767  0.005755  0.014288  0.008373  0.009541              6   
4  0.010454  0.006619  0.016596  0.009507  0.012374              6   

   item_count_12h  
0             3.0  
1             4.0  
2             6.0  
3             2.0  
4             3.0  

[5 rows x 22 columns]

In [14]:
df = extractItemCount(df,whole_qTime,whole_click,4,'item_count_6h')

In [15]:
df.shape

(17383059, 23)

In [16]:
df.head()

user_id  item_id     sim  feature_0     feature_1  feature_2  feature_3  \
0       11    26711  0.0007       50.0  5.338502e+13     0.0012        1.0   
1       11    10528  0.0007       68.0  1.723291e+10     0.0036        1.0   
2       11    22042  0.0007       40.0  4.188502e+14     0.0332        1.0   
3       11    77847  0.0005       30.0  1.547299e+05     0.0168        1.0   
4       11    85475  0.0005       34.0  8.342025e+13     0.0016        1.0   

   feature_4  feature_5  feature_6  ...  feature_10    cn_sim   hpi_sim  \
0     0.0000   490.3025          0  ...    1.634576  0.004169  0.011357   
1     0.0000   472.3559          0  ...    4.357030  0.003971  0.008911   
2     0.2152   487.1326          4  ...    2.254222  0.001825  0.003848   
3     0.0000   472.6520          0  ...    1.695840  0.004337  0.010767   
4     0.0000   485.4152          0  ...    1.603376  0.004169  0.010454   

    hdi_sim  lhn1_sim    ra_sim    aa_sim  time_interval  item_count_12h  \
0  0.010454  0.028477  0.008373  0.010541              6             3.0   
1  0.004760  0.010709  0.009812  0.011679              6             4.0   
2  0.002922  0.006242  0.005647  0.006637              6             6.0   
3  0.005755  0.014288  0.008373  0.009541              6             2.0   
4  0.006619  0.016596  0.009507  0.012374              6             3.0   

   item_count_6h  
0            NaN  
1            NaN  
2            3.0  
3            1.0  
4            1.0  

[5 rows x 23 columns]

In [17]:
df = extractItemCount(df,whole_qTime,whole_click,6,'item_count_4h')

In [18]:
df.shape

(17383059, 24)

In [19]:
df.head()

user_id  item_id     sim  feature_0     feature_1  feature_2  feature_3  \
0       11    26711  0.0007       50.0  5.338502e+13     0.0012        1.0   
1       11    10528  0.0007       68.0  1.723291e+10     0.0036        1.0   
2       11    22042  0.0007       40.0  4.188502e+14     0.0332        1.0   
3       11    77847  0.0005       30.0  1.547299e+05     0.0168        1.0   
4       11    85475  0.0005       34.0  8.342025e+13     0.0016        1.0   

   feature_4  feature_5  feature_6  ...    cn_sim   hpi_sim   hdi_sim  \
0     0.0000   490.3025          0  ...  0.004169  0.011357  0.010454   
1     0.0000   472.3559          0  ...  0.003971  0.008911  0.004760   
2     0.2152   487.1326          4  ...  0.001825  0.003848  0.002922   
3     0.0000   472.6520          0  ...  0.004337  0.010767  0.005755   
4     0.0000   485.4152          0  ...  0.004169  0.010454  0.006619   

   lhn1_sim    ra_sim    aa_sim  time_interval  item_count_12h  item_count_6h  \
0  0.028477  0.008373  0.010541              6             3.0            NaN   
1  0.010709  0.009812  0.011679              6             4.0            NaN   
2  0.006242  0.005647  0.006637              6             6.0            3.0   
3  0.014288  0.008373  0.009541              6             2.0            1.0   
4  0.016596  0.009507  0.012374              6             3.0            1.0   

   item_count_4h  
0            1.0  
1            NaN  
2            1.0  
3            NaN  
4            NaN  

[5 rows x 24 columns]

In [20]:
df = extractItemCount(df,whole_qTime,whole_click,12,'item_count_2h')

In [21]:
df.shape

(17383059, 25)

In [22]:
df.head()

user_id  item_id     sim  feature_0     feature_1  feature_2  feature_3  \
0       11    26711  0.0007       50.0  5.338502e+13     0.0012        1.0   
1       11    10528  0.0007       68.0  1.723291e+10     0.0036        1.0   
2       11    22042  0.0007       40.0  4.188502e+14     0.0332        1.0   
3       11    77847  0.0005       30.0  1.547299e+05     0.0168        1.0   
4       11    85475  0.0005       34.0  8.342025e+13     0.0016        1.0   

   feature_4  feature_5  feature_6  ...   hpi_sim   hdi_sim  lhn1_sim  \
0     0.0000   490.3025          0  ...  0.011357  0.010454  0.028477   
1     0.0000   472.3559          0  ...  0.008911  0.004760  0.010709   
2     0.2152   487.1326          4  ...  0.003848  0.002922  0.006242   
3     0.0000   472.6520          0  ...  0.010767  0.005755  0.014288   
4     0.0000   485.4152          0  ...  0.010454  0.006619  0.016596   

     ra_sim    aa_sim  time_interval  item_count_12h  item_count_6h  \
0  0.008373  0.010541              6             3.0            NaN   
1  0.009812  0.011679              6             4.0            NaN   
2  0.005647  0.006637              6             6.0            3.0   
3  0.008373  0.009541              6             2.0            1.0   
4  0.009507  0.012374              6             3.0            1.0   

   item_count_4h  item_count_2h  
0            1.0            NaN  
1            NaN            NaN  
2            1.0            NaN  
3            NaN            NaN  
4            NaN            NaN  

[5 rows x 25 columns]

In [23]:
df = extractItemCount(df,whole_qTime,whole_click,24,'item_count_1h')

In [24]:
df.shape

(17383059, 26)

In [25]:
df

user_id  item_id     sim  feature_0     feature_1  feature_2  \
0              11    26711  0.0007       50.0  5.338502e+13     0.0012   
1              11    10528  0.0007       68.0  1.723291e+10     0.0036   
2              11    22042  0.0007       40.0  4.188502e+14     0.0332   
3              11    77847  0.0005       30.0  1.547299e+05     0.0168   
4              11    85475  0.0005       34.0  8.342025e+13     0.0016   
...           ...      ...     ...        ...           ...        ...   
17383054    35440    85960  0.0000        1.0  2.022000e-01    11.3570   
17383055    35440    94019  0.0000        2.0  3.000000e-04    88.9092   
17383056    35440      796  0.0000        1.0  2.412000e-01    22.0637   
17383057    35440    17901  0.0000        1.0  1.534000e-01    24.0034   
17383058    35440     9304  0.0000        1.0  1.226000e-01    24.0702   

          feature_3  feature_4  feature_5  feature_6  ...   hdi_sim  lhn1_sim  \
0               1.0     0.0000   490.3025          0  ...  0.010454  0.028477   
1               1.0     0.0000   472.3559          0  ...  0.004760  0.010709   
2               1.0     0.2152   487.1326          4  ...  0.002922  0.006242   
3               1.0     0.0000   472.6520          0  ...  0.005755  0.014288   
4               1.0     0.0000   485.4152          0  ...  0.006619  0.016596   
...             ...        ...        ...        ...  ...       ...       ...   
17383054        1.0     0.1509   261.6800          1  ...  0.000000  0.000000   
17383055       29.0     0.1509   107.8121          1  ...  0.000000  0.000000   
17383056        2.0     0.1509   316.0177          1  ...  0.000000  0.000000   
17383057        4.0     0.1509   128.5136          1  ...  0.000000  0.000000   
17383058        5.0     0.1509   295.2424          1  ...  0.000000  0.000000   

            ra_sim    aa_sim  time_interval  item_count_12h  item_count_6h  \
0         0.008373  0.010541              6             3.0            NaN   
1         0.009812  0.011679              6             4.0            NaN   
2         0.005647  0.006637              6             6.0            3.0   
3         0.008373  0.009541              6             2.0            1.0   
4         0.009507  0.012374              6             3.0            1.0   
...            ...       ...            ...             ...            ...   
17383054  0.000000  0.000000             22             NaN            NaN   
17383055  0.000000  0.000000             22             NaN            NaN   
17383056  0.000000  0.000000             22             1.0            1.0   
17383057  0.000000  0.000000             22             2.0            NaN   
17383058  0.000000  0.000000             22             1.0            NaN   

          item_count_4h  item_count_2h  item_count_1h  
0                   1.0            NaN            NaN  
1                   NaN            NaN            NaN  
2                   1.0            NaN            NaN  
3                   NaN            NaN            NaN  
4                   NaN            NaN            NaN  
...                 ...            ...            ...  
17383054            NaN            NaN            NaN  
17383055            NaN            NaN            NaN  
17383056            NaN            NaN            NaN  
17383057            NaN            NaN            NaN  
17383058            NaN            NaN            NaN  

[17383059 rows x 26 columns]

In [26]:
#df = extractItemCount(df,whole_qTime,whole_click,2,'item_count_12h')
#df = extractItemCount(df,whole_qTime,whole_click,4,'item_count_6h')
#df = extractItemCount(df,whole_qTime,whole_click,6,'item_count_4h')
#df = extractItemCount(df,whole_qTime,whole_click,12,'item_count_2h')
#df = extractItemCount(df,whole_qTime,whole_click,24,'item_count_1h')

In [27]:
df.to_csv('../../user_data/dataset/new_recall/' + file_name + '_additemtime.csv',index=False)